In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
  df = fetch_datasets()['ecoli']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.8, sum(y==1)*.6*.8)
  X[:3]

(336, 7) 2 144.48 16.8


array([[0.49, 0.29, 0.48, 0.5 , 0.56, 0.24, 0.35],
       [0.07, 0.4 , 0.48, 0.5 , 0.54, 0.35, 0.44],
       [0.56, 0.4 , 0.48, 0.5 , 0.49, 0.37, 0.46]])

In [4]:
missing_rates = np.array([0.5, 0.6, 0.7, 0.8, 0.9])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([-90,80])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [5]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [6]:
leftindex = pd.MultiIndex.from_product([["50%", "60%", "70%", "80%", "90%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM         0.627$\pm$0.085   0.885$\pm$0.02   
                         LR          0.792$\pm$0.071  0.856$\pm$0.022   
                         DCT         0.502$\pm$0.193  0.849$\pm$0.035   
             sensitivity SVM          0.55$\pm$0.091   0.87$\pm$0.028   
                         LR          0.746$\pm$0.087  0.827$\pm$0.029   
                         DCT          0.44$\pm$0.187  0.845$\pm$0.043   
60%          F-1 score   SVM          0.65$\pm$0.077  0.884$\pm$0.024   
                         LR          0.812$\pm$0.034  0.856$\pm$0.036   
                         DCT         0.528$\pm$0.231  0.868$\pm$0.038   
             sensitivity SVM         0.573$\pm$0.087  0.865$\pm$0.033   
                         LR          0.764$\pm$0.052   0.824$\pm$0.05   
                         DCT          0.477$\pm$0.22  0.867$\pm$0.029   
70%          F-1 score   SVM         0.666$\pm$0.069   0.83$\pm$0.029   
                         LR          0.783$\pm$0.038  0.821$\pm$0.026   
                         DCT         0.554$\pm$0.289  0.845$\pm$0.027   
             sensitivity SVM         0.589$\pm$0.076    0.8$\pm$0.035   
                         LR          0.727$\pm$0.049  0.779$\pm$0.032   
                         DCT         0.516$\pm$0.242  0.827$\pm$0.037   
80%          F-1 score   SVM         0.674$\pm$0.048  0.855$\pm$0.041   
                         LR          0.763$\pm$0.044  0.829$\pm$0.044   
                         DCT         0.372$\pm$0.213  0.836$\pm$0.056   
             sensitivity SVM         0.602$\pm$0.052  0.847$\pm$0.034   
                         LR          0.707$\pm$0.052   0.796$\pm$0.06   
                         DCT           0.34$\pm$0.16  0.822$\pm$0.059   
90%          F-1 score   SVM         0.684$\pm$0.097  0.833$\pm$0.028   
                         LR          0.705$\pm$0.107  0.802$\pm$0.021   
                         DCT         0.546$\pm$0.233   0.813$\pm$0.06   
             sensitivity SVM          0.618$\pm$0.11   0.84$\pm$0.053   
                         LR          0.644$\pm$0.121  0.782$\pm$0.046   
                         DCT         0.497$\pm$0.201  0.814$\pm$0.093   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM          0.866$\pm$0.03  0.894$\pm$0.032   
                         LR          0.827$\pm$0.036  0.882$\pm$0.027   
                         DCT         0.842$\pm$0.048  0.885$\pm$0.028   
             sensitivity SVM         0.841$\pm$0.037   0.882$\pm$0.04   
                         LR          0.789$\pm$0.049  0.862$\pm$0.036   
                         DCT         0.818$\pm$0.067  0.878$\pm$0.031   
60%          F-1 score   SVM         0.863$\pm$0.027  0.897$\pm$0.035   
                         LR          0.833$\pm$0.027  0.883$\pm$0.031   
                         DCT         0.824$\pm$0.043  0.872$\pm$0.047   
             sensitivity SVM         0.833$\pm$0.035  0.884$\pm$0.042   
                         LR          0.793$\pm$0.036  0.859$\pm$0.044   
                         DCT          0.791$\pm$0.06  0.864$\pm$0.047   
70%          F-1 score   SVM         0.835$\pm$0.019  0.855$\pm$0.024   
                         LR          0.799$\pm$0.031  0.857$\pm$0.031   
                         DCT         0.817$\pm$0.037  0.851$\pm$0.032   
             sensitivity SVM         0.796$\pm$0.024  0.836$\pm$0.033   
                         LR          0.748$\pm$0.039  0.827$\pm$0.037   
                         DCT          0.782$\pm$0.06   0.849$\pm$0.04   
80%          F-1 score   SVM         0.843$\pm$0.034   0.859$\pm$0.04   
                         LR          0.815$\pm$0.035  0.852$\pm$0.038   
                         DCT         0.816$\pm$0.033  0.817$\pm$0.048   
             sensitivity SVM

In [7]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_ecoli_high.csv", index = False)

In [8]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
50\% & F-1 score & SVM &  0.627\$\textbackslash pm\$0.085 &   0.885\$\textbackslash pm\$0.02 &   0.866\$\textbackslash pm\$0.03 &  0.894\$\textbackslash pm\$0.032 &  0.791\$\textbackslash pm\$0.041 &  0.899\$\textbackslash pm\$0.025 \\
    &             & LR &  0.792\$\textbackslash pm\$0.071 &  0.856\$\textbackslash pm\$0.022 &  0.827\$\textbackslash pm\$0.036 &  0.882\$\textbackslash pm\$0.027 &  0.778\$\textbackslash pm\$0.037 &   0.88\$\textbackslash pm\$0.034 \\
    &             & DCT &  0.502\$\textbackslash pm\$0.193 &  0.849\$\textbackslash pm\$0.035 &  0.842\$\textbackslash pm\$0.048 &  0.885\$\textbackslash pm\$0.028 &  0.758\$\textbackslash p